In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [43]:
LABEL_NAME = ["General trash", "Paper", "Paper pack", "Metal", 
              "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

target_csv = '/opt/ml/workspace/ensembled_taekwonV.csv'
output_csv_path = './DUMMY_submission.csv'
df = pd.read_csv(target_csv)
df.head()

,PredictionString,image_id
0,7 0.9993438124656677 607.0755615234375 523.212...,test/0000.jpg
1,5 0.9797833561897278 347.2628173828125 247.238...,test/0001.jpg
2,1 0.9812363386154175 289.4942932128906 305.360...,test/0002.jpg
3,9 0.7993578314781189 166.6297607421875 265.869...,test/0003.jpg
4,1 0.767069399356842 196.54776000976562 259.472...,test/0004.jpg


In [44]:
#Key - Value 쌍은 몇번 id에서 - 몇번 id dummy를 만들 것인가를 의미
dummy_policies = {1 : [0, 2], 5 : [3]}

conf_policies = ((0.3, 0.99), 0.2)

dummy_conf = 0.15

min_conf = conf_policies[0][0]
max_conf = conf_policies[0][1]

In [45]:
predict_strings = df['PredictionString'].tolist()
result_strings = []

for idx, predict_string in enumerate(predict_strings[:]):
    predict_list = str(predict_string).split()
    new_preds = []
    if len(predict_list)==0 or len(predict_list)==1:
        result_strings.append(' ')
        print('!!!')
        continue
    predict_list = np.reshape(predict_list, (-1, 6)).tolist()
    # print(predict_list)
    for pred in predict_list:
        if int(pred[0]) in dummy_policies.keys():
            if min_conf <= float(pred[1]) <= max_conf:
                for dummy_id in dummy_policies[int(pred[0])]:
                    new_preds.append([str(dummy_id), str(dummy_conf), str(pred[2]), str(pred[3]), str(pred[4]), str(pred[5])])
                    # new_preds.append([str(dummy_id)+' '+str(dummy_conf)+' '+pred[2]+' '+pred[3]+' '+pred[4]+' '+pred[5]+' '])
    # print(new_preds)
    predict_list.extend(new_preds)
    predict_list.sort(key = lambda x : (int(x[0]), -float(x[1])))
    # print(predict_list)
    result_string = ''
    for p in predict_list:
        result_string += p[0]+' '+p[1]+' '+p[2]+' '+p[3]+' '+p[4]+' '+p[5]+' '
    result_strings.append(result_string)
    
df['PredictionString'] = result_strings



In [46]:
df.to_csv(output_csv_path, index=None)